In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import talib as ta
warnings.filterwarnings("ignore")
# from Processor import Processor
# 导入相关模块
import pymongo
import pandas as pd

# 连接数据库
client = pymongo.MongoClient('172.16.20.81', 27017)
db = client['Kline_1Min_Auto_Db_Plus']
table = db['eth_btc.spot:binance']
# 读取数据
data = pd.DataFrame(list(table.find()))

In [10]:
df_ethbtc = data.sort_values(by='datetime')[['timestamp','open','high','low','close','volume','datetime']]
dataCoin = df_ethbtc.set_index('datetime')
dataCoin

,timestamp,open,high,low,close,volume
datetime,,,,,,
2018-02-09 17:59:14,1518170354,0.100130,0.100630,0.099245,0.099418,168.6980
2018-02-09 18:00:14,1518170414,0.099418,0.100999,0.099129,0.100298,399.5550
2018-02-09 18:01:14,1518170474,0.100298,0.100908,0.099418,0.099667,286.6310
2018-02-09 18:02:14,1518170534,0.099573,0.100096,0.099531,0.099813,159.2040
2018-02-09 18:03:14,1518170594,0.099968,0.100727,0.099784,0.100096,231.2550
...,...,...,...,...,...,...
2021-12-30 10:47:00,1640832420,0.078285,0.078299,0.078268,0.078293,22.3092
2021-12-30 10:48:00,1640832480,0.078299,0.078300,0.078294,0.078295,38.8840
2021-12-30 10:49:00,1640832540,0.078293,0.078313,0.078279,0.078279,33.5209


In [11]:
dataCoins = []
dataCoins.append(dataCoin.loc['2018-05-01':'2019-05-01'])
dataCoins.append(dataCoin.loc['2019-05-01':'2020-05-01'])
dataCoins.append(dataCoin.loc['2020-05-01':'2021-05-01'])
dataCoins.append(dataCoin.loc['2020-05-01':'2021-09-23'])
dataCoins.append(dataCoin.loc['2021-05-01':'2021-09-23'])
dataCoins.append(dataCoin.loc[:])

In [13]:
def cal_indicator(dataCoin):    
    dataCoin['pctChange'] = dataCoin['close'].pct_change()
    dataCoin['pctStd'] = ta.STDDEV(dataCoin['pctChange'], 24*5)*(24**0.5)
    dataCoin['pctStdVma'] = ta.SUM(dataCoin['pctStd']*dataCoin['volume'], 24*50) / ta.SUM(dataCoin['volume'], 24*50)
    dataCoin['closeVma'] = ta.SUM(dataCoin['close']*dataCoin['volume'], 24*50) / ta.SUM(dataCoin['volume'], 24*50)
#     dataCoin['pctStdVSma'] = ta.EMA(dataCoin.pctStdVma, 24*3)
#     dataCoin['pctStdVLma'] = ta.MA(dataCoin.pctStdVma, 24*10)
for data in dataCoins:
    cal_indicator(data)

In [14]:
def get_stat(df):
    se = df.describe()['pctStd']
    se = se.append(pd.Series(df.skew()['pctStd'],index=['skew']))
    se = se.append(pd.Series(df.kurt()['pctStd'],index=['kurt']))
    se = se.append(pd.Series(se['75%']-se['25%'],index=['75%-25%']))
    return se

In [15]:
ses = []
for data in dataCoins:
    ses.append(get_stat(data))
df = pd.DataFrame(ses)
df = df.T
df.columns = ['2018.5 - 2019.5',
              '2019.5 - 2020.5',
              '2020.5 - 2021.5',
              '2020.5 - 2021.9',
              '2021.5 - 2021.9',
              '2018.5 - 2021.9']
df

,2018.5 - 2019.5,2019.5 - 2020.5,2020.5 - 2021.5,2020.5 - 2021.9,2021.5 - 2021.9,2018.5 - 2021.9
count,525368.000000,526211.000000,525757.000000,734283.000000,209846.000000,2.038942e+06
mean,0.003183,0.003285,0.003345,0.003455,0.003727,3.314810e-03
std,0.001819,0.002194,0.002198,0.002303,0.002520,2.056483e-03
min,0.000000,0.000586,0.000506,0.000506,0.000983,0.000000e+00
25%,0.001956,0.002028,0.002010,0.002155,0.002435,2.072644e-03
50%,0.002737,0.002713,0.002868,0.002957,0.003135,2.852146e-03
75%,0.003851,0.003837,0.004079,0.004133,0.004241,3.975266e-03
max,0.019886,0.040544,0.057600,0.064104,0.064104,6.410442e-02
skew,2.217571,4.726967,5.626188,6.741069,8.566722,5.375903e+00
kurt,8.169183,45.991421,92.663382,119.090288,153.972933,8.528565e+01


In [ ]:
data_over_ma = []
data_below_ma = []
dataCoinsMA = []
dataCoinsMA.append(dataCoin.loc['2018-05-01':'2021-05-01'])
dataCoinsMA.append(dataCoin.loc['2020-05-01':'2021-09-23'])
dataCoinsMA.append(dataCoin.loc['2020-12-01':'2021-04-30'])
for data in dataCoinsMA:
    cal_indicator(data)

for data in dataCoinsMA:
    data_over_ma.append(data[data['close'] >= data['closeVma']])
    data_below_ma.append(data[data['close'] < data['closeVma']])